### ¿Que hace este script?
* Calcula para cada site id: trafico.totales.promedioIN, trafico.totales.promedioOUT, trafico.totales.promedioConsumo

Valores calculado para hora de los ultimos 10 días

In [27]:
from elasticsearch import Elasticsearch, helpers
from ssl import create_default_context
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import parametros
import re

## Conectando a ElasticSearch

La ultima línea se utiliza para garantizar la ejecución de la consulta
* timeout es el tiempo para cada ejecución
* max_retries el número de intentos si la conexión falla
* retry_on_timeout para activar los reitentos

In [28]:
context = create_default_context(cafile=parametros.cafile)
es = Elasticsearch(
    parametros.servidor,
    http_auth=(parametros.usuario_EC, parametros.password_EC),
    scheme="https",
    port=parametros.puerto,
    ssl_context=context,
    timeout=60, max_retries=3, retry_on_timeout=True
) 

### Calculando fechas para la ejecución

* Se calculan las fechas para asociar al nombre del indice
* fecha_hoy es usada para concatenar al nombre del indice principal previa inserción

In [29]:
now = datetime.now()
fecha_hoy = str(now.strftime("%Y.%m.%d"))

### Definiendo indice principal con fecha de hoy

Estos valores se deben ajustar según ambiente. No es automático ya que no hay separación de ambientes

In [30]:
indice = parametros.mintic_concat_index
indice_control = parametros.mintic_control

### Función para generar JSON compatible con ES

In [31]:
def filterKeys(document):
    return {key: document[key] for key in use_these_keys }

### leyendo indice semilla-inventario

En el script que ingesta semilla, trae la información de los centros de conexión administrados. Para el indice principal se requiere:

* site_id como llave del centro de conexión.
* Datos geográficos (Departamento, municipio, centro poblado, sede, energía, latitud, longitud, COD_ISO, entre otros).

In [32]:
total_docs = 10000
try:
    response = es.search(
        index= parametros.semilla_inventario_index,
        body={
               "_source": ['site_id','nombre_municipio', 'nombre_departamento', 'nombre_centro_pob', 'nombreSede' 
                           , 'energiadesc', 'latitud', 'longitud','COD_ISO','id_Beneficiario']
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        source_data = doc["_source"]
        for key, val in source_data.items():
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])

    datos_semilla = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ])) #pd.DataFrame(fields)
except:
    print("fecha:",now,"- Error en lectura de datos semilla")
    #exit()
    

* Se valida latitud y longitud. 
* Se calcula location
* Se renombren campos de semilla

In [33]:
def get_location(x):
    patron = re.compile('^(\-?\d+(\.\d+)?),\s*(\-?\d+(\.\d+)?)$') #patrón que debe cumplir
    if (not patron.match(x) is None):
        return x.replace(',','.')
    else:
        #Código a ejecutar si las coordenadas no son válidas
        return 'a'
datos_semilla['latitud'] = datos_semilla['latitud'].apply(get_location)
datos_semilla['longitud'] = datos_semilla['longitud'].apply(get_location)
datos_semilla['trafico.location'] = datos_semilla['latitud'] + ',' + datos_semilla['longitud']
datos_semilla['trafico.location']=datos_semilla['trafico.location'].str.replace('a,a','')
datos_semilla.drop(columns=['latitud','longitud'],inplace=True)

datos_semilla = datos_semilla.rename(columns={'nombre_municipio': 'trafico.nombreMunicipio'
                                              , 'nombre_departamento' : 'trafico.nombreDepartamento'
                                              , 'nombre_centro_pob': 'trafico.localidad'
                                              , 'nombreSede' : 'trafico.nomCentroDigital'
                                              , 'energiadesc' : 'trafico.sistemaEnergia'
                                              , 'COD_ISO' : 'trafico.codISO'
                                              , 'id_Beneficiario' : 'trafico.idBeneficiario'})
datos_semilla.fillna('', inplace=True)

### leyendo indice cambium-devicedevices

Se lee la información de los dispositivos de red monitoreados por Cambium. En esta lectura no hay referencia de fechas ya que solo hay una ocurrencia por MAC de dispositivo de red.

* site_id es la llave para cruzar con cada centro de conexión.
* mac, IP y name son datos básicos del dispositivo.
* ap_group identifica los dispositivos como INDOOR u OUTDOOR
* status indica el estado del dispositivo en el momento de la lectura.

In [34]:
total_docs = 30000
try:
    response = es.search(
        index= parametros.cambium_d_d_index,
        body={
                    "_source": ["site_id","mac","ap_group"]  
                  , "query": {
                    "match_all": {}
                  }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        source_data = doc["_source"]
        for key, val in source_data.items():
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])

    datos_dev = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ])) #pd.DataFrame(fields)
except:
    exit()

Se descartan registros con site_id vacios y se limpian los NaN del dataframe

In [35]:
datos_dev.dropna(subset=['site_id'])
datos_dev.fillna('', inplace=True)
datos_dev = datos_dev.drop(datos_dev[(datos_dev['site_id']=='')].index)
datos_dev.sort_values(['site_id','ap_group'], inplace=True)

Se limpian datos mal formados de ap_group

In [36]:
datos_dev['ap_group'] = datos_dev['ap_group'].str.split("-", n = 1, expand = True)[0]
datos_dev['ap_group'] = datos_dev['ap_group'].str.split("_", n = 1, expand = True)[0]
datos_dev = datos_dev.drop(datos_dev[(datos_dev['ap_group']=='')].index)

Se borran duplicados por MAC

In [ ]:
datos_dev = datos_dev.drop_duplicates('mac')

Se renombran campos según estructura de indice final

In [37]:
datos_dev = datos_dev.rename(columns={'ap_mac' : 'trafico.macRed','ap_group': 'trafico.apGroup'
                                        , 'mac' : 'trafico.macRed'
                                        , 'status' : 'trafico.status.macRed'})

### Trae la ultima fecha para control de ejecución

Cuando en el rango de tiempo de la ejecución, no se insertan nuevos valores, las fecha maxima en indice mintic no aumenta, por tanto se usa esta fecha de control para garantizar que incremente el bucle de ejecución

In [38]:
total_docs = 1
try:
    response = es.search(
        index= indice_control,
        body={
               "_source": ["trafico.fechaControl"],
              "query": {
                "bool": {
                  "filter": [
                  {
                    "exists": {
                      "field":"jerarquia_trafico_promedios"
                    }
                  }
                  ]
                }
              }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        fecha_ejecucion = doc["_source"]['trafico.fechaControl']
except:
    fecha_ejecucion = '2021-05-01 00:00:00'
if response["hits"]["hits"] == []:
    fecha_ejecucion = '2021-05-01 00:00:00'
print("ultima fecha para control de ejecucion:",fecha_ejecucion)

ultima fecha para control de ejecucion: 2021-06-04 01:00:00


## Se lee la información de cambium device performance

 Se toma los valores de dispositivos y su desempeño.
 * mac del dispositivo de red
 * timestamp es la fecha y hora de la medición
 * radio.* volumen de datos descargados(r) y cargados(t)
 
 Se usa un rango de fechas para calcular promedio y la hora sobre la cual se calculará

In [39]:
def traePerformance(hora,fecha_ini,fecha_tope):
    total_docs = 5000000
    response = es.search(
        index= parametros.cambium_d_p_index,
        body={
                "_source": ["mac","timestamp","radio.5ghz.rx_bps",
                           "radio.5ghz.tx_bps","radio.24ghz.rx_bps"
                          ,"radio.24ghz.tx_bps"]
              , "query": {
                    "bool": {
                      "must": [
                        {
                          "script": {
                            "script": {
                               "source": "doc['timestamp'].value.hour == params.startHour",
                               "params": {
                                 "startHour": hora
                               }
                            }
                          }
                        },
                        {
                          "range": {
                            "timestamp": {
                                "gte": fecha_ini,
                                #"gte": "2021-05-01 00:00:00",
                                "lt": fecha_tope
                            }
                          }
                        }  
                      ]
                    }
                }
        },
        size=total_docs
    )
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        source_data = doc["_source"]
        for key, val in source_data.items():
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])

    return pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ]))

### Lanzando primera ejecución

* Se calcula rango de fechas a procesar. Para este script la muetras sobre la cual se hace el promedio es de los últimos 7 días.
* Se realiza consulta.
* Si no existen datos, se incrementa el rango de fecha y se consulta nuevamente. 
* Se repite el proceso hasta que se consiguen datos o hasta que se alcance la fecha actual.

In [40]:
fecha_max_mintic = fecha_ejecucion
fecha_tope_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(hours=1)-timedelta(seconds=1)).strftime("%Y-%m-%d %H:%M:%S")

fecha_ini = (datetime.strptime(fecha_tope_mintic, '%Y-%m-%d %H:%M:%S')-timedelta(days=7)).strftime("%Y-%m-%d %H:%M:%S")
if fecha_ini[0:10] < '2021-05-01':
    fecha_ini = '2021-05-01 00:00:00'

datos_performance = traePerformance(int(fecha_max_mintic[11:13]),fecha_ini,fecha_tope_mintic)

if datos_performance is None or datos_performance.empty:
    while (datos_performance is None or datos_performance.empty) and ((datetime.strptime(fecha_max_mintic[0:10], '%Y-%m-%d').strftime("%Y-%m-%d %H:%M:%S")) < str(now.strftime("%Y-%m-%d %H:%M:%S"))):
        fecha_max_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(hours=1)).strftime("%Y-%m-%d %H:%M:%S")
        fecha_tope_mintic = (datetime.strptime(fecha_tope_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(hours=1)).strftime("%Y-%m-%d %H:%M:%S")
        fecha_ini = (datetime.strptime(fecha_ini, '%Y-%m-%d %H:%M:%S')+timedelta(hours=1)).strftime("%Y-%m-%d %H:%M:%S")
        datos_performance = traePerformance(int(fecha_max_mintic[11:13]),fecha_ini,fecha_tope_mintic)
else:
    pass

# Insertando promedios a indice

In [42]:
use_these_keys = ['trafico.nomCentroDigital',
                  'trafico.codISO',
                  'trafico.localidad',
                  'trafico.siteID',
                  'trafico.nombreDepartamento',
                  'trafico.sistemaEnergia',
                  'trafico.nombreMunicipio',
                  'trafico.idBeneficiario',
                  'trafico.totales.fechaControl',
                  'trafico.totales.fecha',
                  'trafico.totales.anyo',
                  'trafico.totales.mes',
                  'trafico.totales.dia',
                  'trafico.totales.hora',
                  'trafico.totales.promedioIN',
                  'trafico.totales.promedioOUT',
                  'trafico.totales.promedioConsumo',
                    'nombreDepartamento',
                    'nombreMunicipio',
                    'idBeneficiario',
                    'fecha',
                    'anyo',
                    'mes',
                    'dia',
                  '@timestamp']
def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": indice, 
                "_id": f"{'PromedioTra-' + document['trafico.siteID'] + '-' +document['trafico.totales.fechaControl']}",
                "_source": filterKeys(document),
            }

Calculo de indicadores de consumos
* Se hace promedio por sitio y fecha

In [43]:
try:
    datos_performance.drop_duplicates(inplace=True)
    datos_performance.rename(columns={'mac': 'trafico.macRed'}, inplace=True)
    datos_performance = pd.merge(datos_performance, datos_dev, on='trafico.macRed',how='inner')
    datos_performance.replace('','0',inplace=True)
    datos_performance.fillna({'radio.5ghz.rx_bps':0,
                      'radio.5ghz.tx_bps':0,
                      'radio.24ghz.rx_bps':0,
                      'radio.24ghz.tx_bps':0  
                      },inplace=True)
    datos_performance[['radio.5ghz.rx_bps','radio.5ghz.tx_bps','radio.24ghz.rx_bps','radio.24ghz.tx_bps']] = datos_performance[['radio.5ghz.rx_bps','radio.5ghz.tx_bps','radio.24ghz.rx_bps','radio.24ghz.tx_bps']].astype(float)
    datos_performance['trafico.totales.traficoIN_aux'] = datos_performance['radio.5ghz.rx_bps'] + datos_performance['radio.24ghz.rx_bps']
    datos_performance['trafico.totales.traficoOUT_aux'] = datos_performance['radio.5ghz.tx_bps'] + datos_performance['radio.24ghz.tx_bps']
    aux_performance=datos_performance[['site_id'
                      , 'trafico.totales.traficoIN_aux'
                      ,'trafico.totales.traficoOUT_aux']].groupby(['site_id']).agg(['mean']).reset_index()
    aux_performance.columns = aux_performance.columns.droplevel(1)
    aux_performance['trafico.totales.promedioIN'] = round((aux_performance['trafico.totales.traficoIN_aux']/float(1<<30)),6)
    aux_performance['trafico.totales.promedioOUT'] = round((aux_performance['trafico.totales.traficoOUT_aux']/float(1<<30)),6)
    aux_performance['trafico.totales.promedioConsumo'] = aux_performance['trafico.totales.promedioIN'] + aux_performance['trafico.totales.promedioOUT']
    #Se totaliza para la sede completa
    aux_performance['trafico.totales.promedioConsumo'] = round(aux_performance['trafico.totales.promedioConsumo'],6)

    ### Generando columnas con fecha, anyo, mes, dia, hora y minuto por separado
    aux_performance["trafico.totales.fechaControl"] = fecha_max_mintic
    aux_performance["trafico.totales.fecha"] = fecha_max_mintic.split(" ")[0]
    aux_performance["trafico.totales.hora"] = fecha_max_mintic.split(" ")[1].split(":")[0]
    aux_performance["trafico.totales.anyo"] = fecha_max_mintic[0:4]
    aux_performance["trafico.totales.mes"] = fecha_max_mintic[5:7]
    aux_performance["trafico.totales.dia"] = fecha_max_mintic[8:10]
    ### Renombrado de campos

    ##nulos a cero
    aux_performance.fillna({'trafico.totales.promedioConsumo':0,
                      'trafico.totales.promedioIN':0,
                      'trafico.totales.promedioOUT':0
                      },inplace=True)
    aux_performance[['trafico.totales.promedioConsumo','trafico.totales.promedioIN','trafico.totales.promedioOUT']] = aux_performance[['trafico.totales.promedioConsumo','trafico.totales.promedioIN','trafico.totales.promedioOUT']].astype(float)
    mintic_02 = pd.merge(datos_semilla,  aux_performance, on='site_id',how='inner')
    mintic_02.rename(columns={'site_id': 'trafico.siteID'}, inplace=True)
    mintic_02['@timestamp'] = now.isoformat()
    mintic_02['nombreDepartamento'] = mintic_02['trafico.nombreDepartamento']
    mintic_02['nombreMunicipio'] = mintic_02['trafico.nombreMunicipio']
    mintic_02['idBeneficiario'] = mintic_02['trafico.idBeneficiario']
    mintic_02['fecha'] = mintic_02['trafico.totales.fecha']
    mintic_02['anyo'] = mintic_02['trafico.totales.anyo']
    mintic_02['mes'] = mintic_02['trafico.totales.mes']
    mintic_02['dia'] = mintic_02['trafico.totales.dia']
    salida = helpers.bulk(es, doc_generator(mintic_02))
    print("Fecha: ", now,"- Datos Trafico promedios en indice principal:",salida[0])
except:
    print("Fecha: ", now,"- No se insertaron datos de promedios en indice principal")   

Fecha:  2021-06-10 15:34:29.113392 - Datos Trafico promedios en indice principal: 117


### Guardando fecha para control de ejecución

* Se actualiza la fecha de control. Si el calculo supera la fecha hora actual, se asocia esta ultima.

In [ ]:
fecha_ejecucion = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(hours=1)).strftime("%Y-%m-%d %H:%M:%S")

if fecha_ejecucion > str(now.strftime('%Y-%m-%d %H:%M:%S'))[0:14] + '00:00':
    fecha_ejecucion = str(now.strftime('%Y-%m-%d %H:%M:%S'))[0:14] + '00:00'
response = es.index(
        index = indice_control,
        id = 'jerarquia_trafico_promedios',
        body = { 'jerarquia_trafico_promedios': 'trafico_promedios','trafico.fechaControl' : fecha_ejecucion}
)
print("actualizada fecha control de ejecucion:",fecha_ejecucion)